In [70]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [71]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Title

In [72]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFK', #'FKfpi', 'FpiFpi', 'FKFK'
    'bias_correct' : True,
    
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
}

In [75]:
for vol in [0, 10]:
    for include_alphaS in [False, True]:
        for include_logSq in [False, True]:
            
            t0 = time.time()
            
            # Loop through combinations
            p_dict['order']['vol'] = vol
            p_dict['order']['include_log'] = include_alphaS
            p_dict['order']['include_log2'] = include_logSq
            
            if vol > 0:
                include_vol = True
            else:
                include_vol = False

            # Load data
            data_loader = dl.data_loader()
            fit_data = data_loader.get_fit_data()

            
            bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], 
                                           fit_type=p_dict['fit_type'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'])
            
            new_prior = bootstrapper.create_prior_from_fit()
            data_loader.save_prior(new_prior, fit_type=p_dict['fit_type'], F2=p_dict['F2'], 
                                   include_FV=include_FV, include_alphaS=include_alphaS, include_logSq=include_logSq)
            
            t1 = time.time()

            print "(Vol, alphaS, logSq):", (include_vol, include_alphaS, include_logSq)
            print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
Done!
100% complete Time (s):  0.799999952316
Compiling results...
Saving...
Done.

Total time (s):  33.103000164 

Making fits...
Done!
100% complete Time (s):  0.84500002861
Compiling results...
Saving...
Done.

Total time (s):  32.1129999161 

Making fits...
Done!
100% complete Time (s):  0.926000118256
Compiling results...
Saving...
Done.

Total time (s):  32.6930000782 



KeyboardInterrupt: 